In [ ]:
import sys
import pdb
import gym
import math
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from tensorboardX import SummaryWriter
from BayesianNetwork import BayesianNetwork
from BayesianQNetwork import BQN_learn

from Chain_env import chain_env

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
envt = "Chain"
def action_modify(act):
    if envt == 'Chain':
        act = 2*act - 1
    return act

In [ ]:
def test_agent(agent):
    
    count = 0    
    test_episode_rew = 0
    test_return = []
    
    done = False
    agent.dbqn.eval()
    
    obs = env.reset()
    act = agent.act(obs, use_sample=True, num_sample=4)
    
    while count <= 99:
        if done:
            test_return.append(test_episode_rew)
            test_episode_rew = 0
            count = count + 1
            
            obs = env.reset()
            act = agent.act(obs, use_sample=True, num_sample=4)
        
        obs1, rew, done, _ = env.step(action_modify(act))        
        act = agent.act(obs1, use_sample=True, num_sample=0)                
        test_episode_rew = test_episode_rew + rew        
    
    agent.dbqn.train()
    return np.mean(np.array(test_return))

In [ ]:
batch_size = 64
lr = 1e-2
N = 25
gamma = 1
env = chain_env(N)       
steps = 50000
train_start = 1000
buffer_size = 50000
features_list = [N, 32, 2]

In [ ]:
N_arr = [25, 50, 75, 100]
runs = len(N_arr)
run_result = []
solved_dict = {}

for run in range(3, runs):
        
    N = N_arr[run]
    env = chain_env(N)
    features_list = [N, 32, 2]
    writer = SummaryWriter('results/Chain/run' + str(run))
    dbqn = BayesianNetwork(features_list, 4, batch_size, steps).to(DEVICE)    
    target_dbqn = BayesianNetwork(features_list, 4, batch_size, steps).to(DEVICE)
    agent = BQN_learn(dbqn, target_dbqn, gamma, lr, batch_size, buffer_size, writer, train_start)
    
    done = False
    solved = 0
    episode_rew = 0
    episode_count = 0
    res = []

    obs = env.reset()
    act = agent.reset(obs)         

    max_state = 1
    while agent.t <= steps and episode_count < 1000:
        
        if done:
            print("Episode " + str(episode_count) + " with reward = " + str(episode_rew) + " max state reached = "
                 + str(max_state))  
            writer.add_scalar('data/reward', episode_rew, episode_count)
            writer.add_scalar('data/max_state', max_state, episode_count)
            res.append(episode_rew)
            if episode_rew >= 10:
                solved = solved + 1
            
            episode_rew = 0
            episode_count = episode_count + 1                            
                        
            max_state = 1
            if episode_count%25 == 0:
                test_result = test_agent(agent)
                print("Test Result = " + str(test_result))
                writer.add_scalar('data/test_reward', test_result, episode_count)
                
            if solved == 100:
                solved_dict[N] = episode_count - 1
                break
                

            obs = env.reset()
            act = agent.reset(obs)   

        obs1, rew, done, _ = env.step(action_modify(act))
        act = agent.step(obs, act, rew, obs1, done)
        obs = obs1
        episode_rew = episode_rew + rew  
        if env.current_state > max_state:
            max_state = env.current_state        
                
    writer.close()        
    run_result.append(res)